## Image Annotation Tool
Annotate bounding boxes or points

In [ ]:
import numpy as np
import random
import os
import cv2 as cv
import screeninfo
from tqdm import tqdm
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--datadir', type=str, required=True, help='The directory of image files')
parser.add_argument('--mode', type=str, default='rect', choices=['rect', 'point'])
parser.add_argument('--samples', type=int, default=50, help='number of samples to label')

args = parser.parse_args()

drawing = False # true if mouse is pressed
mode = True # if True, draw rectangle. Press 'm' to toggle to curve
ix,iy = -1,-1
xx, yy = -1, -1
# mouse callback function
def draw_rect(event,x,y,flags,param):
    assert args.mode == 'rect'
    global ix,iy,drawing,mode, draw, xx, yy
    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        ix,iy = x,y
    elif event == cv.EVENT_MOUSEMOVE:
        if drawing == True:
            draw = img.copy()
            cv.rectangle(draw,(ix,iy),(x,y),(0,255,0), 1, 8, 0)
            # cv.circle(draw,(x,y),5,(0,0,255),-1)
            # cv.putText(draw, f'{ix}, {iy}', (img.shape[1]-50, 10), cv.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0, 255, 0))
            # cv.putText(draw, f'{x}, {y}', (img.shape[1]-50, 20), cv.FONT_HERSHEY_SIMPLEX, fontScale=0.3, color=(0, 255, 0))
            xx, yy = x, y
    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        # if mode == True:
        #     cv.rectangle(draw,(ix,iy),(x,y),(0,255,0), 1, 8, 0)
        # else:
        #     cv.circle(draw,(x,y),5,(0,0,255),-1)
        
def mouse_click(event,x,y,flags,param):
    global ix,iy,drawing,mode, draw, xx, yy
    if event == cv.EVENT_LBUTTONDOWN:
        draw = img.copy()
        cv.circle(draw, (x, y), radius=4, color=(0, 255, 0), thickness=-1)
        xx, yy = x, y
        
img_files = [f for f in os.listdir(args.datadir) if f.endswith('.tiff')]
random.seed(123)
random.shuffle(img_files)
IMG_WIDTH, IMG_HEIGHT = 40, 20
IMG_MAX_DIM = max(IMG_WIDTH, IMG_HEIGHT)
SCALE_FACTOR = 10
screen = screeninfo.get_monitors()[0]
screen_width, screen_height = screen.width, screen.height
for img_file in tqdm(img_files[:args.samples]):
    img = cv.imread(os.path.join(args.datadir, img_file))
    assert img.shape[0] == 20
    assert img.shape[1] == 40
    if args.mode == 'rect':
        anno_file = os.path.join(args.datadir, img_file.replace('.tiff', '_rect.txt'))
    else:
        anno_file = os.path.join(args.datadir, img_file.replace('.tiff', '_center.txt'))
    # resize image
    img = cv.resize(img, (img.shape[1]*SCALE_FACTOR, img.shape[0]*SCALE_FACTOR), interpolation = cv.INTER_LANCZOS4)
    draw = img.copy()
    cv.namedWindow(img_file)
    # Move the window to screen center
    cv.moveWindow(img_file, screen_width//2, screen_height//2)
    if args.mode == 'rect':    
        cv.setMouseCallback(img_file, draw_rect)
    elif args.mode == 'point':
        cv.setMouseCallback(img_file, mouse_click)
        
    # Load and draw existing annotation
    if os.path.exists(anno_file):
        with open(anno_file, 'r') as f:
            if args.mode == 'rect':
                data = list(map(float, f.readline().split(',')))
                ix, iy, xx, yy = (np.array(data)*IMG_MAX_DIM*SCALE_FACTOR).astype(np.int)
                cv.rectangle(draw,(ix,iy),(xx,yy),(0,255,0), 1, 8, 0)
            elif args.mode == 'point':
                xx, yy = map(float, f.readline().split(','))
                cv.circle(draw, (int(xx*IMG_MAX_DIM*SCALE_FACTOR), int(yy*IMG_MAX_DIM*SCALE_FACTOR)), 
                          radius=4, color=(0, 255, 0), thickness=-1)
	
    while(1):
        cv.imshow(img_file,draw)
        k = cv.waitKey(1) & 0xFF
        if k == ord('m'):
            mode = not mode
        # Press ESC to save annotation and continue
        elif k == 27: 
            with open(anno_file, 'w') as f:
                if args.mode == 'rect':
                    ix, iy = ix/IMG_MAX_DIM/SCALE_FACTOR, iy/IMG_MAX_DIM/SCALE_FACTOR
                    xx, yy = xx/IMG_MAX_DIM/SCALE_FACTOR, yy/IMG_MAX_DIM/SCALE_FACTOR
                    f.write(f'{ix}, {iy}, {xx}, {yy}')
                elif args.mode == 'point':
                    xx, yy = xx/IMG_MAX_DIM/SCALE_FACTOR, yy/IMG_MAX_DIM/SCALE_FACTOR
                    f.write(f'{xx}, {yy}')
            break
    cv.destroyAllWindows()
